# Analysis of CA1 neurons during the autopi task

The analysis depends on 2 main packages developped in the lab.

* spikeA
* autopypi


## Rename positrack2 files

The positrack2 software has the following name scheme: mouse-date-time.positrack2
We want to have mouse-date_01.positrack2, mouse-date_02.positrack2, etc

* rename_positrack2_files.ipynb

## Power spectra 

To make sure that the raw .dat files are not corrupted, I will create a power spectrum with the beginning and end of each .dat files. 

* power_spectra_inspection.ipynb

## Channel mapping

We test whether the channel mapping makes sense using a correlation matrix of raw signals from different channels. The assumption is that channels in close proximity should be more correlated than further away channels.

* channels_correlation_matrix.ipynb

## Database tests

Before we start, we want to make sure we have a database with all the needed files.

* database_test.ipynb



## Extraction of lever position

We need to extract the lever position for further analysis. We can do this with a deeplabcut network applied to the arena video.

We use the same procedure as for the behavioral data set.

* position_tracking.ipynb

It takes approximately 30 minutes per session to perform the analysis with a RTX 2080 ti NVIDIA GPU.

## Improve position tracking using off-line algorithm and additional arena camera

The online tracking algorithm is set to minimize false positive at all cost because the data are then passed to the cable actuator. 

For some sessions with positrack2, we have the recorded video that can be used to improve the tracking that was done online. 


* dlc_on_positrack2_videos.ipynb -> offline tracking for some sessions with video
* kalman_filter.ipynb -> apply kalman filter to remove noise and complete path when small stretches are missing


## Spatial transformation of position data and temporal transformation from ROS time to electrophysiology time.

We used 2 different camera systems during the autopi task with electrophysiology. The Positrack software tracked infrared LEDs and we also recorded a video of the arena to know the lever position.

We transformed the position data so that these two systems are aligned. 

The rule is that...

* 0,0 is the center of the arena
* The home base relative to the center of the arena is in the direction of 0,-1 vector. 
* The values are in cm.

There are also 2 different reference times. The recording times (duration within ktan .dat files) and ROS time (computers clock).

To do the analysis, it is simpler if all the data are in the same spatial and temporal reference frame, or if they have both times.

The animal pose time is usually using the recording time, we added the ROS time for all position data. We save the ROS time in the `Animal_pose.pose` matrix (7th column).

The lever position is also calculated for all row of the `Animal_pose.pose` matrix. This way, we can get the lever position for each trial.

For positrack data, we do not see the arena so we will do the detection manually. There are approximately 50 sessions.

* positrack_data_transform.ipynb


We have position data that were filtered using a Kalman filter in `pose_kf.npy` and unfiltered data in `pose.npy`. The filtered data should be used to analyse the behavior on the autopi task.


## Lever pose file to go with animal pose.

* lever_pose.ipynb


## Bridge position in arena reference frame (cm)

We need to get the bridge position in the arena-center reference frame. The coordinates should be in cm

We will use `arenaCoordinates` and `bridgeCoordinates`, and generate `bridgeCoordinatesCm`

* bridge_coordinates_cm.ipynb

## Check head-direction against heading in the path

We want to make sure that 0, pi/2, pi, and -pi/2 means the same thing in all our sessions. For instance, does 0 refer to East, North, West or South? We need to have the same reference frame throughout the experiment. 2 tracking systems were used so it is important to check.

* check_head_direction_heading.ipynb

## Identify channels with sharp-wave ripples (SWR) and the pyramidal cell layer.

SWR are seen only in the pyramidal cell layer of the hippocampus.
We can use their presence to identify which shanks of the probes were in the pyramidal cell layer.
We detected SWRs from the last rest trial of the session programatically.
Then we performed a visual inspection of SWR events to determine which shanks were in the pyramidal cell layer.

* sharp_wave_ripples_detection.ipynb

We stored the results in `shank_layer.csv` file, whith shankNo and layer as header. This file is later used to select neurons from the pyramidal cell layer

## Get mean spike waveforms of clusters

This gets the waveform on the channel with the largest amplitude. The channel with the largest amplitude is also saved in order to assign a shank to each neuron.

* mean_waveform.ipynb

## Identify the shank on which a neuron was recorded

* shank_of_neurons.ipynb


## Classification as pyramidal cells or interneurons

* cell_type_classification2.ipynb

This notebook generates 2 supplementary figures and `/adata/projects/autopi_ca1/results/cell_classification.csv` that can be used later on.


## Creation of the trial table

Create a table containing  autopy trials start and end time and the light condition. The time is in ROS and electrophysiological time. 

This trial table can be loaded for later analysis without always having to recalculate these.

* trial_table_simple.ipynb

It is called "simple" because the trials are not yet decomposed in journeys.

A file called `trial_table_simple` is created in each session directory.

More detailed trial extraction below


## Trials, journeys and NavPath extraction

Here we extract trials, journeys and NavPath for each session.

We create 2 DataFrames

* One summarizing each NavPath of the session
* One with the instantaneous behavioral data within each NavPath

The strategy for the data analysis is that we will get the instantaneous firing rate of neurons that can be matched to the instantaneous behavioral data (time in path, distance run, x position, y position, etc). 

We also save the myProject object with the trial extraction done so that we can reload it later without re-running the analysis all the time.

* extract_trial.ipynb

Output
* /adata/projects/autopi_ca1/results/myProjectWithTrials.pickle


## Maps of open field, light trials and dark trials

To get an idea of the firing properties of neurons during light and dark trails, we create firing rate maps of open field, light and dark trials.

* **open_field_light_dark_maps.ipynb**

Creates

* /adata/projects/autopi_ca1/results/myAutos.pickle
* /adata/projects/autopi_ca1/results/myMaps.pickle


## Compare maps between open field, light and dark trials

The main aim is to study the difference between the open field and the task, and between light and dark trials. 

We use standard place cell analysis that most readers will want to see (mean rate, peak rate, info scores, map correlations)

We also look at cell pairs IFR correlation and map similarity for different environments.

This is not going into the details of trials. 

* **light_dark_stats.ipynb**

Creates

* /adata/projects/autopi_ca1/results/mapCorDf.csv
* /adata/projects/autopi_ca1/results/myMapStats.csv


* **light_dark_stats_pairs.ipynb** (IRF and map similarity of cell pairs)


* **NOT USED ANYMORE: light_dark_stats_figure.ipynb** Figure introducing place cell



# Maps of search and homing path for light and dark trials

To get an idea of the firing properties of neurons during light and dark trails, we create firing rate maps of open field, light and dark trials.

* **search_homing_light_dark_maps.ipynb**
* **search_homing_stats.ipynb**
* **search_homing_stats_pair.ipynb**

Creates

* /adata/projects/autopi_ca1/results/mySearchHomingMaps.pickle
* /adata/projects/autopi_ca1/results/mySearchHomingMapStats.csv
* /adata/projects/autopi_ca1/results/searchHomingMapCorDf.csv


Analysis with figure
* **NOT USED ANYMORE search_homing_figure.ipynb**

# Merge comparison of random foraging with task and search/homing paths into one figure

This uses the data generated in the two sections above and make a figure with it.

It is copied from `light_dark_stats_figure.ipynb` and `search_homing_figure.ipynb`

* **remapping_figure.ipynb**

## Instantaneous firing rates

We create a matrix containing the instantaneous firing rate of all neurons at 50Hz. The data is stored in a file in each session directory.

These will be correlated to the behavioral variables later on.

* instantaneous_firing_rate.ipynb

## Correlation between instantaneous firing rate and NavPath variables 

* ifr_behav_exploration.ipynb




## Search and Homing path analysis

We need to find out what control the activity of pyramidal cells when the animal is "searching" for the lever.

* **search_path_data_collection.ipynb**
* **homing_path_data_collection.ipynb**


* **search_homing_trial_matrix.ipynb**

The notebook above calculate the trial rate matrix internal correlation



* search_path_model_fields.ipynb # not used yet in figure
* search_path_fast_slow_paths.ipynb # not successful yet

Current figure 
* OLD: search_path_figure_identify_fields.ipynb
* **search_homing_path_figure.ipynb**

## Lever-centered maps

We can check if some neurons have a nice firing field locked on the lever position. We would expect this to be stronger in the light when the animal sees the lever.

* lever_centered_maps.ipynb

This notebook creates many data files including:


* /adata/projects/autopi_ca1/results/myLeverStats.csv
* /adata/projects/autopi_ca1/results/myLeverMaps.pickle
* /adata/projects/autopi_ca1/results/myLeverStatsThreshold.csv
* /adata/projects/autopi_ca1/results/dfLev.csv

The following notebook create figures 

* lever_centered_maps_figure.ipynb

## Lever-centered maps related to search path and homing error

We have firing feilds around the lever that are lock to the cardinal reference frame. 

Are these fields related to the properties of the search path?
Are these fields related to homing accuracy?

We have one jupyter notebook to generate the maps we need.
 * **lever_centered_sh.ipynb** 

And one jupyter notebook to do stats and figures.
* **lever_centered_sh_figure.ipynb**

## Head direction selectivity

We want to compare head-direction coding during open field, light and dark condition.

* open_field_light_dark_hd_histo.ipynb

This notebook calculates all the HD firing rate histograms (and respective firing rate maps). We also get the stats related to HD coding. 
The results are saved in 

* /adata/projects/autopi_ca1/results/myDirMaps.pickle, dictionary with key per condition
* /adata/projects/autopi_ca1/results/dirMapCorDf.csv, data frame with correlation of north and south maps
* /adata/projects/autopi_ca1/results/myHdHist.pickle, dictionary with key per condition
* /adata/projects/autopi_ca1/results/myHdStats.csv, data frame with hd stats
* /adata/projects/autopi_ca1/results/myHdStability.csv, data frame with stability (r) for across different conditions (oo,ol,od,ld)
* /adata/projects/autopi_ca1/results/myHdThreshold.csv, threshold for the vector length


There are some figures in the notebook to test the sanity of the analysis.

* light_dark_hd_figure.ipynb


## Homing vector coding

Test whether some hippocampal neurons encode information the difference between heading and homing direction. 

* homing_vector_histo.ipynb

This notebook calculate egocentric homing angle firing rate histograms and the stats of these histograms

* /results/myEhaHist.pickle
* /results/myEhaStats.csv


We are comparing HD and EHA coding in the following notebook

* homing_vector_histo_figure.ipynb


## Plot the path on the mouse for each trial of a session

We try to plot the path of the mouse for each trial on the task.

* plot_single_trial_path.ipynb